In [ ]:
HTML(read(open("style.css"), String))

# Memoization

This notebook contains alle functions needed for memoizing function calls that we have already calculated. This will greatly reduce the runtime of the program.

The general idea is implemented in this function:
1. If the value has been calculated: Look up the value in the global Cache using the arguments of the function as the key and return that value.
1. If the value has not been calculated: Calculate the value as usual and before returning the value add the value to the global Cache for later lookup.

```
function memoize(f)
    global gCache
    function f_memoized(b, s, d)    # TODO: replace (b, s, d) with args... to make function generic
        if (b, s, d) in keys(gCache)
            return gCache[args]
        end
        result = f(b, s, d)
        merge!(gCache, Dict((b, s, d) => result))
        return result
    end
    return f_memoized
end
```

For Minimax we can use this simple idea of Memoization.

In [ ]:
using Chess
using NBInclude

In [ ]:
@nbinclude("ZobristHashing.ipynb")

The function `initCache()` returns an empty Dictionary of type `Dict{UInt64, Tuple{String, Int64, Int64}}` which will be used for the global Cache variable. As the key we use the hash of a board. The value consists of a tuple consisting of 3 arguments:
1. `flag::String` is "=", "<=" or ">=" representing if the value stored is the exact value or a upper or lower boundary
1. `value::Int64` is the value of the position
1. `depth::Int64` is the maximum depth at which the value has been calculated at.

In [ ]:
function initCache()
    return Dict{UInt64, Tuple{String, Int64, Int64}}()
end

## Memoization with Alpha-Beta-Pruning

When calculating with Alpha-Beta-Pruning we cannot use the same idea of using the arguments as the keys. If we were to use this method the key would consist of the following tuple:

$f:(hash, value, depth, alpha, beta) \longrightarrow (value)$ 

The issue with this is that the values for alpha and beta are constantly changing during the evaluation process. As the function needs an exact hit on the cache, meaning all arguments are the same, this will rarely be the case. Therefore the total amount of cache hits will be very low in comparison to the Minimax algorithm. 

Therefore we need a new strategy. We use the facts of the alpha-beta-algorithm and save whether the stored value is an exact value or if it has been pruned. If it is pruned we use an `<=` or `>=` flag to state that the memoized value is at least or at most the value we save in the cache.

If the value stored v is:
1. An exact value:

    $\alpha < v < \beta \longrightarrow cache[hash] = ("=", value, depth)$
    
1. below the lower boundary $\alpha$:

    $v <= \alpha \longrightarrow cache[hash] = ("<=", value, depth)$
    
1. above the upper boundary $\beta$:

    $\beta < v \longrightarrow cache[hash] = (">=", value, depth)$

This way we have achieved implementing Memoization for alpha-beta-search. We use the function `evaluate` to wrap the Memoization around a given function `f`(for alphaBetaMax and alphaBetaMin). Additionally we use an auxilliary function `store_cache`to store new values into the cache using the method above.


The `evaluate` function adds memoization to any function using the alpha-beta-pruning algorithm. 
This function is used to evaluate the given board state using the provided evaluation function (f) and cache. It performs alpha-beta pruning to reduce the number of nodes that need to be evaluated, and checks the cache to see if the given state has already been evaluated before.

Arguments:
1. `aBoard::AdvBoard` is the current state
1. `f::Function` takes in either the function that needs to be calculated and memoized
1. `depth::Int64` is the number of halfmoves the engine should analyze before terminating
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}` is the cache used
1. `alpha::Int64` is a minimal value that has been calculated during the recursive process
1. `beta::Int64`  is a maximal value that has been calculated during the recursive process
1. `flagQuiesce::Bool` is an optional flag specifies whether the quiesce-search should be used.

The function `evaluate` evaluates the same result as the function `f`. Additionally it saves calculated results in the `cache`. And uses any entries in the `cache` if the same function has already been called.

In [ ]:
function evaluate(aBoard::AdvBoard, f::Function, depth::Int64, cache::Dict{UInt64, Tuple{String, Int64, Int64}},
                  alpha::Int64, beta::Int64, flagQuiesce::Bool=false)::Int64
    
    tuple::Tuple{String, Int64, Int64} = get(cache, aBoard.hash, ("", 0, 0))
    if tuple != ("", 0, 0)
        flag::String, v::Int64, d::Int64 = tuple
        if d >= depth 
            if ((flag == "=") || (flag == "<=" && v <= alpha) || (flag == ">=" && beta <= v))
                return v
            elseif flag == "<="
                beta = min(beta, v)
            elseif flag == ">="
                alpha = max(alpha, v)
            end
        end
    end
    # no value stored in gCache for State or depth of stored State has less depth than required
    if flagQuiesce
        v = f(aBoard, depth, cache, alpha, beta, flagQuiesce)
    else
        v = f(aBoard, depth, cache, alpha, beta)
    end
    store_cache(aBoard.hash, depth, cache, alpha, beta, v)
    return v
end

The `store_cache` is a helping function that stores values into the `cache`. 

Arguments:
1. `hash`  is the hash that needs to be stored
1. `depth` is the number of halfmoves the engine should analyze before terminating
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}` is the cache used
1. `alpha` is a minimal value that has been calculated during the recursive process
1. `beta`  is a maximal value that has been calculated during the recursive process
1. `v` is the value that got calculated by the `evaluate` function that needs to be stored


In [ ]:
function store_cache(hash::UInt64, depth::Int64, 
                     cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64, beta::Int64, v::Int64)
    if v <= alpha
        push!(cache, hash => ("<=", v, depth))
    elseif v < beta
        push!(cache, hash => ("=", v, depth))
    else # beta <= v
        push!(cache, hash => (">=", v, depth))
    end
end